In [ ]:
!pip install --upgrade gensim
!pip install transformers
!pip install -U sentence-transformers
!pip install kornia

Requirement already up-to-date: gensim in /home/jamunoz/anaconda3/lib/python3.8/site-packages (4.0.1)


Requirement already up-to-date: sentence-transformers in /home/jamunoz/anaconda3/lib/python3.8/site-packages (1.1.1)


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, hamming_loss, confusion_matrix, jaccard_score
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import transformers
from transformers import RobertaTokenizer, BertTokenizer, RobertaModel, BertModel, AdamW
from transformers import get_linear_schedule_with_warmup
import time
from kornia.losses import BinaryFocalLossWithLogits

import warnings
import traceback

In [ ]:
!nvidia-smi

Tue May 18 22:55:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 2080    Off  | 00000000:03:00.0 Off |                  N/A |
|  0%   42C    P0    39W / 260W |      3MiB /  7982MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 2080    Off  | 00000000:05:00.0 Off |                  N/A |
|  0%   

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:1")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(1))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: GeForce RTX 2080


##Preparing Dataset

###Load dataset

In [ ]:
categories_number_words = {
        1: "Apoyo Pedagógico en asignaturas",
        3: "Apoyo pedagógico personal",
        4: "Tutoría entre pares",
        7: "Hacer a la familia partícipe del proceso",
        8: "Apoyo psicóloga(o)",
        9: "Apoyo fonoaudióloga(o)",
        10: "Apoyo Educador(a) Diferencial",
        11: "Apoyo Kinesióloga(o)",
        12: "Apoyo Médico General",
        13: "Apoyo Terapeuta Ocupacional",
        14: "Control Neurólogo",
        15: "Apoyo Interdisciplinario",
        16: "Adecuación curricular de acceso",
        17: "Adecuación curricular de objetivos"
    }
categories_words_number = {v: k for k, v in categories_number_words.items()}

diagnoses_codes = {
    "Trastorno específico del lenguaje": 0,
    "Trastorno por déficit atencional": 1,
    "Dificultad específica de aprendizaje": 2,
    "Discapacidad intelectual": 3,
    "Discapacidad visual": 4,
    "Trastorno del espectro autista": 5,
    "Discapacidad auditiva - Hipoacusia": 6,
    "Funcionamiento intelectual limítrofe": 7,
    "Síndrome de Down": 8,
    "Trastorno motor": 9,
    "Multidéficit": 10,
    "Retraso global del desarrollo": 11
}

diagnoses_keys = list(diagnoses_codes.keys())

def transform_diag_to_array(code):
    arr = np.zeros(len(diagnoses_keys), dtype=int)
    for (index, label) in enumerate(diagnoses_keys):
        if diagnoses_codes[label]==code:
            arr[index] = 1
    return arr

In [ ]:
train_dataset = pd.read_csv('/research/jamunoz/datasets/train_ds.csv', keep_default_na=False)
val_dataset = pd.read_csv('/research/jamunoz/datasets/val_ds.csv', keep_default_na=False)
test_dataset = pd.read_csv('/research/jamunoz/datasets/test_ds.csv', keep_default_na=False)
# train_dataset = pd.read_csv('gdrive/My Drive/magister/train_ds.csv', keep_default_na=False)
# val_dataset = pd.read_csv('gdrive/My Drive/magister/val_ds.csv', keep_default_na=False)
# test_dataset = pd.read_csv('gdrive/My Drive/magister/test_ds.csv', keep_default_na=False)

# Add OHE diagnosis
train_OHE_diags = []
for diag in train_dataset['Encoded Diagnosis']:
    train_OHE_diags.append(transform_diag_to_array(diag))
temp_train_diags_df = pd.DataFrame(train_OHE_diags, columns=diagnoses_keys)
train_dataset = pd.concat([train_dataset, temp_train_diags_df], axis=1)

val_OHE_diags = []
for diag in val_dataset['Encoded Diagnosis']:
    val_OHE_diags.append(transform_diag_to_array(diag))
temp_val_diags_df = pd.DataFrame(val_OHE_diags, columns=diagnoses_keys)
val_dataset = pd.concat([val_dataset, temp_val_diags_df], axis=1)

test_OHE_diags = []
for diag in test_dataset['Encoded Diagnosis']:
    test_OHE_diags.append(transform_diag_to_array(diag))
temp_test_diags_df = pd.DataFrame(test_OHE_diags, columns=diagnoses_keys)
test_dataset = pd.concat([test_dataset, temp_test_diags_df], axis=1)

In [ ]:
train_dataset.head()

In [ ]:
# y_keys = list(strat_present.keys())
Y_KEYS = list(categories_words_number.keys())

# df = pd.DataFrame(data=new_dataset_to_export)
# X = df
# Y = df[y_keys]
X_train = train_dataset.drop(Y_KEYS, axis=1)
Y_train = train_dataset[Y_KEYS]
X_val = val_dataset.drop(Y_KEYS, axis=1)
Y_val = val_dataset[Y_KEYS]
X_test = test_dataset.drop(Y_KEYS, axis=1)
Y_test = test_dataset[Y_KEYS]

strats_amounts = {
              'Adecuación curricular de acceso': 2264,
              'Hacer a la familia partícipe del proceso': 2048,
              'Apoyo Interdisciplinario': 1441, 
              'Apoyo Educador(a) Diferencial': 1311,
              'Apoyo pedagógico personal': 1240,
              'Apoyo fonoaudióloga(o)': 378,
              'Apoyo psicóloga(o)': 588,
              'Apoyo Terapeuta Ocupacional': 153,
              'Tutoría entre pares': 350,
              'Control Neurólogo': 63,
              'Apoyo Médico General': 64,
              'Apoyo Kinesióloga(o)': 32,
              'Adecuación curricular de objetivos': 281,
              'Apoyo Pedagógico en asignaturas': 1314
}
most_unbalanced_strategies = [strategy for strategy in Y_KEYS if (
    strats_amounts[strategy] < (len(X_train) + len(X_val) + len(X_test))*0.15 or strats_amounts[strategy] > (len(X_train) + len(X_val) + len(X_test))*0.85)]
less_unbalanced_strategies = [strategy for strategy in Y_KEYS if strategy not in most_unbalanced_strategies]
only_one_strat = [Y_KEYS[0]]

In [ ]:
Y_train.keys()

###Data explore

In [ ]:
print('Positive rates:')
for label in list(Y_train.keys()):
    print(label+': ', round(sum(Y_train[label])/len(Y_train[label]), 2))

Positive rates:
Apoyo Pedagógico en asignaturas:  0.43
Apoyo pedagógico personal:  0.41
Tutoría entre pares:  0.11
Hacer a la familia partícipe del proceso:  0.67
Apoyo psicóloga(o):  0.19
Apoyo fonoaudióloga(o):  0.12
Apoyo Educador(a) Diferencial:  0.43
Apoyo Kinesióloga(o):  0.01
Apoyo Médico General:  0.02
Apoyo Terapeuta Ocupacional:  0.05
Control Neurólogo:  0.02
Apoyo Interdisciplinario:  0.47
Adecuación curricular de acceso:  0.73
Adecuación curricular de objetivos:  0.09


In [ ]:
percs = X_train['All perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
print(0 in seq_len)

False


In [ ]:
percs = X_train['Special Education Teacher Perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
print(0 in seq_len)

True


In [ ]:
percs = X_train['Speech Therapist Perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
percs = X_train['Psychologist Perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
percs = X_train['Medical Perceptions']
seq_len = [len(i.split()) for i in percs]

pd.Series(seq_len).hist(bins = 30)

###Dataset

In [ ]:
class AllJoinedObservationsDataset(Dataset):

    def __init__(self, data, tokenizer, chunk_len=200, overlap_len=50, approach="all"):
        self.tokenizer = tokenizer
        self.chunk_len = chunk_len
        self.overlap_len = overlap_len
        self.approach = approach
        self.data = data

    def __len__(self):
        return len(self.data)

    def long_terms_tokenizer(self, tokenized_data, labels):
        long_terms_token = []
        input_ids_list = []
        attention_mask_list = []
        token_type_ids_list = []
        labels_list = []

        previous_input_ids = tokenized_data["input_ids"].reshape(-1)
        previous_attention_mask = tokenized_data["attention_mask"].reshape(-1)
        previous_token_type_ids = tokenized_data["token_type_ids"].reshape(-1)
        remain = tokenized_data.get("overflowing_tokens").reshape(-1)
        labels = torch.tensor(labels, dtype=torch.int)

        input_ids_list.append(previous_input_ids)
        attention_mask_list.append(previous_attention_mask)
        token_type_ids_list.append(previous_token_type_ids)
        labels_list.append(labels)

        if remain.size() and self.approach != 'head':
            remain = torch.tensor(remain, dtype=torch.long)
            idxs = range(len(remain)+self.chunk_len)
            idxs = idxs[(self.chunk_len-self.overlap_len-2)
                         ::(self.chunk_len-self.overlap_len-2)]
            input_ids_first_overlap = previous_input_ids[-(
                self.overlap_len+1):-1]
            start_token = torch.tensor([101], dtype=torch.long)
            end_token = torch.tensor([102], dtype=torch.long)

            for i, idx in enumerate(idxs):
                if i == 0:
                    input_ids = torch.cat(
                        (input_ids_first_overlap, remain[:idx]))
                elif i == len(idxs):
                    input_ids = remain[idx:]
                elif previous_idx >= len(remain):
                    break
                else:
                    input_ids = remain[(previous_idx-self.overlap_len):idx]

                previous_idx = idx

                nb_token = len(input_ids)+2
                attention_mask = torch.ones(self.chunk_len, dtype=torch.long)
                attention_mask[nb_token:self.chunk_len] = 0
                token_type_ids = torch.zeros(self.chunk_len, dtype=torch.long)
                input_ids = torch.cat((start_token, input_ids, end_token))
                if self.chunk_len-nb_token > 0:
                    padding = torch.zeros(
                        self.chunk_len-nb_token, dtype=torch.long)
                    input_ids = torch.cat((input_ids, padding))

                input_ids_list.append(input_ids)
                attention_mask_list.append(attention_mask)
                token_type_ids_list.append(token_type_ids)
                labels_list.append(labels)
            if self.approach == "tail":
                input_ids_list = [input_ids_list[-1]]
                attention_mask_list = [attention_mask_list[-1]]
                token_type_ids_list = [token_type_ids_list[-1]]
                labels_list = [labels_list[-1]]

        return({
            'ids': input_ids_list,  # torch.tensor(ids, dtype=torch.long),
            # torch.tensor(mask, dtype=torch.long),
            'mask': attention_mask_list,
            # torch.tensor(token_type_ids, dtype=torch.long),
            'token_type_ids': token_type_ids_list,
            'labels': labels_list,
            'len': [torch.tensor(len(labels_list), dtype=torch.long)]
        })


    def __getitem__(self, idx):
      data_row = self.data.iloc[idx]
      labels =  data_row[Y_KEYS]

      data = self.tokenizer.encode_plus(
            data_row['All perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
      
      long_token = self.long_terms_tokenizer(data, labels)

      tensor_labels = torch.tensor(labels, dtype=torch.int)
      tensor_diags = torch.tensor(data_row[diagnoses_keys], dtype=torch.int)

      dict_to_return = dict(
          text_data = long_token,
          labels=tensor_labels,
          diagnostics=tensor_diags
      )

      return dict_to_return

In [ ]:
class SeparatedObservationsDataset(Dataset):

    def __init__(self, data, tokenizer, chunk_len=200, overlap_len=50, approach="all"):
        self.tokenizer = tokenizer
        self.chunk_len = chunk_len
        self.overlap_len = overlap_len
        self.approach = approach
        self.data = data

    def __len__(self):
        return len(self.data)

    def long_terms_tokenizer(self, tokenized_data, labels):
        long_terms_token = []
        input_ids_list = []
        attention_mask_list = []
        token_type_ids_list = []
        labels_list = []

        previous_input_ids = tokenized_data["input_ids"].reshape(-1)
        previous_attention_mask = tokenized_data["attention_mask"].reshape(-1)
        previous_token_type_ids = tokenized_data["token_type_ids"].reshape(-1)
        remain = tokenized_data.get("overflowing_tokens").reshape(-1)
        labels = torch.tensor(labels, dtype=torch.int)

        input_ids_list.append(previous_input_ids)
        attention_mask_list.append(previous_attention_mask)
        token_type_ids_list.append(previous_token_type_ids)
        labels_list.append(labels)

        if remain.size() and self.approach != 'head':
            remain = torch.tensor(remain, dtype=torch.long)
            idxs = range(len(remain)+self.chunk_len)
            idxs = idxs[(self.chunk_len-self.overlap_len-2)
                         ::(self.chunk_len-self.overlap_len-2)]
            input_ids_first_overlap = previous_input_ids[-(
                self.overlap_len+1):-1]
            start_token = torch.tensor([101], dtype=torch.long)
            end_token = torch.tensor([102], dtype=torch.long)

            for i, idx in enumerate(idxs):
                if i == 0:
                    input_ids = torch.cat(
                        (input_ids_first_overlap, remain[:idx]))
                elif i == len(idxs):
                    input_ids = remain[idx:]
                elif previous_idx >= len(remain):
                    break
                else:
                    input_ids = remain[(previous_idx-self.overlap_len):idx]

                previous_idx = idx

                nb_token = len(input_ids)+2
                attention_mask = torch.ones(self.chunk_len, dtype=torch.long)
                attention_mask[nb_token:self.chunk_len] = 0
                token_type_ids = torch.zeros(self.chunk_len, dtype=torch.long)
                input_ids = torch.cat((start_token, input_ids, end_token))
                if self.chunk_len-nb_token > 0:
                    padding = torch.zeros(
                        self.chunk_len-nb_token, dtype=torch.long)
                    input_ids = torch.cat((input_ids, padding))

                input_ids_list.append(input_ids)
                attention_mask_list.append(attention_mask)
                token_type_ids_list.append(token_type_ids)
                labels_list.append(labels)
            if self.approach == "tail":
                input_ids_list = [input_ids_list[-1]]
                attention_mask_list = [attention_mask_list[-1]]
                token_type_ids_list = [token_type_ids_list[-1]]
                labels_list = [labels_list[-1]]

        return({
            'ids': input_ids_list,  # torch.tensor(ids, dtype=torch.long),
            # torch.tensor(mask, dtype=torch.long),
            'mask': attention_mask_list,
            # torch.tensor(token_type_ids, dtype=torch.long),
            'token_type_ids': token_type_ids_list,
            'labels': labels_list,
            'len': [torch.tensor(len(labels_list), dtype=torch.long)]
        })


    def __getitem__(self, idx):
      data_row = self.data.iloc[idx]
      labels =  data_row[Y_KEYS]

      set_data = self.tokenizer.encode_plus(
            data_row['Special Education Teacher Perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
    
      st_data = self.tokenizer.encode_plus(
            data_row['Speech Therapist Perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
        
      p_data = self.tokenizer.encode_plus(
            data_row['Psychologist Perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
        
      m_data = self.tokenizer.encode_plus(
            data_row['Medical Perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
      
      set_long_token = self.long_terms_tokenizer(set_data, labels)      
      st_long_token = self.long_terms_tokenizer(st_data, labels)      
      p_long_token = self.long_terms_tokenizer(p_data, labels)      
      m_long_token = self.long_terms_tokenizer(m_data, labels)

      tensor_labels = torch.tensor(labels, dtype=torch.int)
      tensor_diags = torch.tensor(data_row[diagnoses_keys], dtype=torch.int)

      dict_to_return = dict(
          text_data_set = set_long_token,
          text_data_st = st_long_token,
          text_data_p = p_long_token,
          text_data_m = m_long_token,
          labels=tensor_labels,
          diagnostics=tensor_diags
      )

      return dict_to_return

###Dataloaders

In [ ]:
TRAIN_BATCH_SIZE=1
EPOCH=6
CHUNK_LEN=150
OVERLAP_LEN=30
GRADIENT_ACCUMULATIONS = 16

print('Loading BERT tokenizer...')
bert_tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", do_lower_case=True)

def my_collate1(batches):
  modified_batches = []
  for batch in batches:
    batch_dict = {}
    for key, value in batch.items():
      if key == "text_data":
        new_dict = {}
        for sub_key, sub_value in batch[key].items():
          new_dict[sub_key] = torch.stack(sub_value)
        batch_dict[key] = new_dict
      else:
        batch_dict[key] = value
    modified_batches.append(batch_dict)
  return modified_batches

transformed_train_dataset=AllJoinedObservationsDataset(
    train_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

transformed_val_dataset=AllJoinedObservationsDataset(
    val_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

transformed_test_dataset=AllJoinedObservationsDataset(
    test_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

train_data_loader=DataLoader(
    transformed_train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate1)

val_data_loader=DataLoader(
    transformed_val_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate1)

test_data_loader=DataLoader(
    transformed_test_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate1)

############ separated observations ############

def my_collate2(batches):
  modified_batches = []
  for batch in batches:
    batch_dict = {}
    for key, value in batch.items():
      if key == "text_data_set" or key == "text_data_st" or key == "text_data_p" or key == "text_data_m":
        new_dict = {}
        for sub_key, sub_value in batch[key].items():
          new_dict[sub_key] = torch.stack(sub_value)
        batch_dict[key] = new_dict
      else:
        batch_dict[key] = value
    modified_batches.append(batch_dict)
  return modified_batches

divided_transformed_train_dataset=SeparatedObservationsDataset(
    train_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

divided_transformed_val_dataset=SeparatedObservationsDataset(
    val_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

divided_transformed_test_dataset=SeparatedObservationsDataset(
    test_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

divided_train_data_loader=DataLoader(
    divided_transformed_train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate2)

divided_val_data_loader=DataLoader(
    divided_transformed_val_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate2)

divided_test_data_loader=DataLoader(
    divided_transformed_test_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate2)

Loading BERT tokenizer...


###Fine-tunning BERT

In [ ]:
class Bert_FT_Model(nn.Module):
    """ A Model for bert fine tuning """

    def __init__(self, n_classes):
        super(Bert_FT_Model, self).__init__()
        self.bert_path = "dccuchile/bert-base-spanish-wwm-cased"
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.out = nn.Linear(768, n_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, ids, mask, token_type_ids):
        """ Define how to perfom each call
        Parameters
        __________
        ids: array
            -
        mask: array
            - 
        token_type_ids: array
            -
        Returns
        _______
            - 
        """
        _, output = self.bert(
            ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        output = self.relu(output)
        output = self.dropout(output)
        output = self.out(output)
        # output = torch.sigmoid(output)
        return output
    
    
class Single_Bert_FT_Model(nn.Module):
    """ A Model for bert fine tuning """

    def __init__(self):
        super(Single_Bert_FT_Model, self).__init__()
        self.bert_path = "dccuchile/bert-base-spanish-wwm-cased"
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.out = nn.Linear(768, 1)
        # self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.3)

    def forward(self, ids, mask, token_type_ids, is_eval=False):
        """ Define how to perfom each call
        Parameters
        __________
        ids: array
            -
        mask: array
            - 
        token_type_ids: array
            -
        Returns
        _______
            - 
        """
        _, output = self.bert(
            ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        if is_eval:
          print('representacion: ', output)
        # output = self.relu(output)
        # output = self.dropout(output)
        output = self.out(output)
        # output = torch.sigmoid(output)
        return output

In [ ]:
class Bert_Hierarchical_Model(nn.Module):

    def __init__(self, n_classes, pooling_method="mean"):
        super(Bert_Hierarchical_Model, self).__init__()

        self.pooling_method = pooling_method

        self.bert_path = "dccuchile/bert-base-spanish-wwm-cased"
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.out = nn.Linear(768, n_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, ids, mask, token_type_ids, lengt):

        _, pooled_out = self.bert(
            ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)

        chunks_emb = pooled_out.split_with_sizes(lengt)

        if self.pooling_method == "mean":
            emb_pool = torch.stack([torch.mean(x, 0) for x in chunks_emb])
        elif self.pooling_method == "max":
            emb_pool = torch.stack([torch.max(x, 0)[0] for x in chunks_emb])

        output = self.relu(emb_pool)
        output = self.dropout(output)
        output = self.out(output)
        # output = torch.sigmoid(output)

        return output
    

class Single_Bert_Hierarchical_Model(nn.Module):

    def __init__(self, pooling_method="mean"):
        super(Single_Bert_Hierarchical_Model, self).__init__()

        self.pooling_method = pooling_method

        self.bert_path = "dccuchile/bert-base-spanish-wwm-cased"
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.out = nn.Linear(768, 1)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.3)

    def forward(self, ids, mask, token_type_ids, lengt):

        _, pooled_out = self.bert(
            ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)

        chunks_emb = pooled_out.split_with_sizes(lengt)

        if self.pooling_method == "mean":
            output = torch.stack([torch.mean(x, 0) for x in chunks_emb])
        elif self.pooling_method == "max":
            output = torch.stack([torch.max(x, 0)[0] for x in chunks_emb])

#         output = self.relu(output)
#         output = self.dropout(output)
        output = self.out(output)
        # output = torch.sigmoid(output)

        return output

In [ ]:
import statistics

def get_results(targets, outputs):
  TN = 0
  TP = 0
  FP = 0
  FN = 0
  for (i, output) in enumerate(outputs):
    if output==0:
      if targets[i]==0:
        TN += 1
      else:
        FN += 1
    else:
      if targets[i]==1:
        TP += 1
      else:
        FP += 1
  return TP, TN, FP, FN

def findMinDiff(arr):
    n = len(arr)
    arr = sorted(arr)
    diff = 0.5
    for i in range(n-1):
        if arr[i+1] - arr[i] > 0 and arr[i+1] - arr[i] < diff:
            diff = arr[i+1] - arr[i]
    return diff

def get_thresholds(targets, outputs):
  best_thresholds = []
  for i in range(len(outputs[0])):
    real_preds = outputs[:, i]
    trues = targets[:, i]
    max_g = 0
#     max_f1 = 0
    delta_threshold = 0.0001 # findMinDiff(real_preds)*0.9
    positive_ratio = sum(trues)/len(trues)
#     print('pr: ', positive_ratio)
    if positive_ratio > 0.6:
      local_best = 0
      curr_threshold = min(real_preds)
#       print(curr_threshold)
      while curr_threshold < 1:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
#         f1 = f1_score(trues, preds)
#         print(f1, max_f1, curr_threshold, local_best, tp, tn, fp, fn)
        if tp < tn:
          break
        if g_mean > max_g:
#         if f1 > max_f1:
          max_g = g_mean
#           max_f1 = f1
          local_best = curr_threshold
        curr_threshold += delta_threshold
      best_thresholds.append(local_best)
    elif positive_ratio < 0.4:
      local_best = 1
      curr_threshold = max(real_preds)
      while curr_threshold > 0:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
#         f1 = f1_score(trues, preds)
        if tn < tp:
          break
        if g_mean > max_g:
#         if f1 > max_f1:
          max_g = g_mean
#           max_f1 = f1
          local_best = curr_threshold
        curr_threshold -= delta_threshold
      best_thresholds.append(local_best)
    else:
      local_best = 0.5
      best_thresholds.append(local_best)
  return best_thresholds

def get_individual_threshold(target, output):
    real_preds = output
    trues = target
    max_g = 0
    # max_f1 = 0
    delta_threshold = 0.0001 # findMinDiff(real_preds)*0.9
    positive_ratio = sum(trues)/len(trues)
#     print('pr: ', positive_ratio)
    if positive_ratio > 0.5:
      local_best = 0
      curr_threshold = min(real_preds)
#       print(curr_threshold)
      while curr_threshold < 1:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
        # f1 = f1_score(trues, preds)
#         print(f1, max_f1, curr_threshold, local_best, tp, tn, fp, fn)
        if tp < tn:
          break
        if g_mean > max_g:
        # if f1 > max_f1:
          max_g = g_mean
          # max_f1 = f1
          local_best = curr_threshold
        curr_threshold += delta_threshold
      return local_best
    else:
      local_best = 1
      curr_threshold = max(real_preds)
      while curr_threshold > 0:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
        # f1 = f1_score(trues, preds)
        if tn < tp:
          break
        if g_mean > max_g:
        # if f1 > max_f1:
          max_g = g_mean
          # max_f1 = f1
          local_best = curr_threshold
        curr_threshold -= delta_threshold
      return local_best
    # else:
    #   local_best = 0.5
    #   return local_best
  

In [ ]:
def loss_fun(outputs, targets):
    loss = nn.BCEWithLogitsLoss()
    # loss = BinaryFocalLossWithLogits(alpha=0.25, reduction='mean')
    try:
      return loss(outputs, targets)
    except Exception:
      print(outputs, targets)
      traceback.print_exc()
    # return nn.BCEWithLogitsLoss()(outputs, targets)

def individual_evaluation(target, predicted):
  individual = {}
  for i in range(len(target[0])):
    temp_t = target[:, i]
    temp_p = predicted[:, i]
    diction = dict(
        accuracy=accuracy_score(temp_t, temp_p),
        f1=f1_score(temp_t, temp_p)
    )
    individual[str(i)] = diction
  return individual

def evaluate(target, predicted):
    thresholds = get_thresholds(target, predicted)
    print('thresholds: ', thresholds)
    true_predicted = np.array([[1 if val > thresholds[i] else 0 for (i, val) in enumerate(pred)] for pred in predicted])
    accuracy = accuracy_score(target, true_predicted)
    macro_f1 = f1_score(target, true_predicted, average='macro')
    micro_f1 = f1_score(target, true_predicted, average='micro')
    weighted_f1 = f1_score(target, true_predicted, average='weighted')
    hl = hamming_loss(target, true_predicted)
    js = jaccard_score(target, true_predicted)
    macro_js = jaccard_score(target, true_predicted, average="macro")
    micro_js = jaccard_score(target, true_predicted, average="micro")
    individual = individual_evaluation(target, true_predicted)
    return {
        "accuracy": accuracy,
        "jaccard_score_average": js,
        "jaccard_score_macro": macro_js,
        "jaccard_score_micro": micro_js,
        "macro-f1": macro_f1,
        "micro-f1": micro_f1,
        "Hamming Loss": hl,
        "Individual": individual
    }

def individual_evaluation(target, predicted):
    threshold = get_individual_threshold(target, predicted)
    print('threshold: ',threshold)
    true_predicted = np.array([1 if val > threshold else 0 for val in predicted])
    default_true_predicted = np.array([1 if val > 0.5 else 0 for val in predicted])
    accuracy = accuracy_score(target, true_predicted)
    f1 = f1_score(target, true_predicted)
    tp, tn, fp, fn = get_results(target, true_predicted)
    recall = tp/(tp+fn)
    specificity = tn/(tn+fp)
    g_mean = np.sqrt(recall*specificity)
    pr = sum(target)/len(target)

    default_accuracy = accuracy_score(target, default_true_predicted)
    default_f1 = f1_score(target, default_true_predicted)
    tp, tn, fp, fn = get_results(target, default_true_predicted)
    default_recall = tp/(tp+fn)
    default_specificity = tn/(tn+fp)
    default_g_mean = np.sqrt(default_recall*default_specificity)
    return {
        "Positive Rate": pr,
        "accuracy": accuracy,
        "f1": f1,
        "recall": recall,
        "specificity": specificity,
        "g_mean": g_mean,
        "default_accuracy": default_accuracy,
        "default_f1": default_f1,
        "default_recall": default_recall,
        "default_specificity": default_specificity,
        "default_g_mean": default_g_mean
    }

In [ ]:
def train_loop_fun1(data_loader, model, optimizer, device, grad_accs, scheduler=None):
    model.train()
    t0 = time.time()
    losses = []
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        targets = [data["text_data"]["labels"] for data in batch]
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fun(outputs, targets)
        (loss / grad_accs).backward()
        model.float()
        if (batch_idx + 1) % grad_accs == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
        losses.append(loss.item())
        if batch_idx % 250 == 0:
            print(
                f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} seconds ___")
            t0 = time.time()
    return losses


def individual_train_loop_fun1(data_loader, model, optimizer, device, grad_accs, scheduler=None, label_index=0):
    model.train()
    t0 = time.time()
    losses = []
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["text_data"]["labels"][:, label_index] for data in batch]
        targets = []
        for label_set in labels:
          miniset = []
          for label in label_set:
            miniset.append(torch.tensor([label]))
          targets.append(torch.stack(miniset))
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        # outputs = torch.cat(torch.unbind(outputs))
        loss = loss_fun(outputs, targets)
        (loss / grad_accs).backward()
        model.float()
        if (batch_idx + 1) % grad_accs == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
        losses.append(loss.item())
        if batch_idx % 250 == 0:
            print(
                f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} seconds ___")
            t0 = time.time()
    return losses

In [ ]:
def eval_loop_fun1(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        targets = [data["text_data"]["labels"] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        with torch.no_grad():
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())

        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

def individual_eval_loop_fun1(data_loader, model, device, label_index=0):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["text_data"]["labels"][:, label_index] for data in batch]
        targets = []
        for label_set in labels:
          miniset = []
          for label in label_set:
            miniset.append(torch.tensor([label]))
          targets.append(torch.stack(miniset))

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        with torch.no_grad():
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            # outputs = torch.cat(torch.unbind(outputs))
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())

        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    # print(fin_outputs)
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

In [ ]:
# best_global_f1 = 0
# best_global_model = 0
# best_i = 0

learning_rates_results = {
    0: {
        "1_6": 0.5325153374233128,
        "5_6": 0.5848269089123497,
        "10_6": 0.5285329414646093,
        "15_6": "destroyed"
  },
    1: {
        "1_6": 0.5084662576687116,
        "5_6": 0.5188017845761632,
        "10_6": 0.5148916967509025,
        "15_6": "destroyed",
    },
    2: {
        "1_6": 0.19258167526406286,
        "5_6": 0.18746964545896067,
        "10_6": "destroyed"
    },
    3: {
        "1_6": 0.43559488692232046,
        "5_6": 0.4472324723247232,
        "10_6": 0.42899408284023666,
        "15_6": 0.3855297157622739,
        "20_6": 0.44406196213425125,
        "25_6": "destroyed"
    },
    4: {
        "1_6": 0.34560628375061364,
        "5_6": "destroyed"
    },
    5: {
        "1_6": 0.19293078055964655,
        "5_6": "destroyed"
    },
    6: {
        "1_6": 0.5096980112938866,
        "5_6": 0.5283757338551859,
        "10_6": "destroyed"
    },
    7: {
        "1_6": 0.02995335133808004,
        "5_6": 0.029568589432864757,
        "10_6": 0.029872673849167482,
        "15_6": "destroyed"
    },
    8: {
        "1_6": 0.0343980343980344,
        "5_6": 0.034778349253000246,
        "10_6": 0.034407559970923186,
        "15_6": 0.03457511565619674,
        "20_6": "destroyed"
    },
    9: {
        "1_6": 0.09423312883435583,
        "5_6": 0.09205819222513711,
        "10_6": "destroyed"
    },
    10: {
        "1_6": 0.04637539663168171,
        "5_6": 0.0464774951076321,
        "10_6": 0.045820131611016335,
        "15_6": 0.045728038507821894,
        "20_6": "destroyed"
    },
    11: {
        "1_6": 0.5153223829369944,
        "5_6": 0.5170635894917751,
        "10_6": 0.5230844793713163,
        "15_6": 0.4995098039215686,
        "20_6": 0.5038910505836575,
        "25_6": "destroyed"
    },
    12: {
        "1_6": 0.3500491642084562,
        "5_6": 0.35111876075731496,
        "10_6": 0.34808259587020646,
        "15_6": 0.35013523481681824,
        "20_6": 0.3544739429695182,
        "25_6": 0.348574237954769
    },
    13: {
        "1_6": 0.16213568454567717,
        "5_6": 0.16129032258064518,
        "10_6": 0.1601996823235761,
        "15_6": "destroyed"
    }

}

learning_rates_coeffs = {
    0: 5,
    1: 5,
    2: 1,
    3: 5,
    4: 1,
    5: 1,
    6: 5,
    7: 1,
    8: 5,
    9: 1,
    10: 10,
    11: 10,
    12: 20,
    13: 1
}


MULTI_LABELS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

for lab in MULTI_LABELS:
    LABEL_INDEX = lab
    i = learning_rates_coeffs[LABEL_INDEX]

    print('#################################################### Para label: ', Y_KEYS[LABEL_INDEX], " ("+str(LABEL_INDEX)+") ##############")
    # i = 20

    print("------------------------------------------- exp: "+str(i)+"*1e-6 -----------------------------------------")
    lr=(i)*1e-6 # Algunos lr hacen que BERT se destruya 
    num_training_steps=int(len(transformed_train_dataset) / TRAIN_BATCH_SIZE * EPOCH)

    # Modify according to individual or all
    # if i > 0:
    #     del model
    #     del best_model
    model=Single_Bert_FT_Model().to(device) # Bert_FT_Model(len(Y_KEYS)).to(device)
    optimizer=AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                          num_warmup_steps=0,
                                          num_training_steps=num_training_steps)
    val_losses=[]
    batches_losses=[]
    val_acc=[]

    best_model = None
    best_g_mean = 0

    for epoch in range(EPOCH):
      t0 = time.time()    
      print(f"\n=============== EPOCH {epoch+1} / {EPOCH} ===============\n")
      # Modify according to individual or all
      batches_losses_tmp=individual_train_loop_fun1(train_data_loader, model, optimizer, device, GRADIENT_ACCUMULATIONS, scheduler=scheduler, label_index=LABEL_INDEX)
      epoch_loss=np.mean(batches_losses_tmp)
      print(f"\n*** avg_loss : {epoch_loss:.2f}, time : ~{(time.time()-t0)//60} min ({time.time()-t0:.2f} sec) ***\n")
      t1=time.time()
      # Modify according to individual or all
      output, target, val_losses_tmp=individual_eval_loop_fun1(val_data_loader, model, device, label_index=LABEL_INDEX)
      print(f"==> evaluation : avg_loss = {np.mean(val_losses_tmp):.2f}, time : {time.time()-t1:.2f} sec\n")
      # Modify according to individual or all
      tmp_evaluate=individual_evaluation(target, output)
      if tmp_evaluate['g_mean'] > best_g_mean or tmp_evaluate['default_g_mean'] > best_g_mean:# and tmp_evaluate['recall'] != 0 and tmp_evaluate['specificity'] != 0:
        del best_model
        torch.cuda.empty_cache()
        best_model = model
        if tmp_evaluate['g_mean'] > tmp_evaluate['default_g_mean']:
            best_g_mean = tmp_evaluate['g_mean']
        else:
            best_g_mean = tmp_evaluate['default_g_mean']
      print(f"=====>\t{tmp_evaluate}")
      val_acc.append(tmp_evaluate['accuracy'])
      val_losses.append(val_losses_tmp)
      batches_losses.append(batches_losses_tmp)
    torch.save(best_model, f"/research/jamunoz/models/fine_tuned_bert/model_"+str(lab)+".pt")
    del model, optimizer, scheduler, best_model
    torch.cuda.empty_cache()
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

# if best_model:
#     if best_f1 > best_global_f1:
#       best_global_f1 = best_f1
#       best_global_model = best_model
#       best_i = i
#       print('hay uno mejor')
#     else:
#       print('hay uno bueno pero no lo suficiente')
# else:
#     print('murió para este: ', lr)
# if best_global_model:
#   torch.save(best_global_model, f"/research/jamunoz/models/fine_tuned_bert/model_"+str(LABEL_INDEX)+"__f1_"+str(best_global_f1)+"__lr_"+str(best_i)+"_10-6.pt")
#   print("\t§§ model has been saved §§")
# else:
#   print("como que murió el modelo")

#################################################### Para label:  Apoyo pedagógico personal  (1) ##############
------------------------------------------- exp: 5*1e-6 -----------------------------------------


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



=============== EPOCH 1 / 6 ===============

___ batch index = 0 / 1836 (0.00%), loss = 0.8387, time = 0.19 seconds ___


/home/jamunoz/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2016: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-18-5fb9fe6293fb>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  remain = torch.tensor(remain, dtype=torch.long)


...

___ batch index = 1750 / 1836 (95.32%), loss = 0.0752, time = 29.82 seconds ___



*** avg_loss : 0.33, time : ~3.0 min (209.10 sec) ***



==> evaluation : avg_loss = 0.32, time : 28.74 sec



threshold:  [0.05761987]
=====>	{'Positive Rate': array([0.08699926], dtype=float32), 'accuracy': 0.15753256328336201, 'f1': 0.1593918587542913, 'recall': 0.9180790960451978, 'specificity': 0.08506056527590848, 'g_mean': 0.2794500436171009, 'default_accuracy': 0.9130007372818875, 'default_f1': 0.0, 'default_recall': 0.0, 'default_specificity': 1.0, 'default_g_mean': 0.0}


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [ ]:
# lista = list(enumerate(val_data_loader))

In [ ]:
# labels = [data["labels"][LABEL_INDEX] for data in lista[0][1]]
# targets = []
# if len(labels) > 1:
#     for label_set in labels:
#       miniset = []
#       for label in label_set:
#         miniset.append(torch.tensor([label]))
#       targets.append(torch.stack(miniset))
# else:
#     miniset = [torch.tensor(labels[0])]
#     targets.append(torch.stack(miniset))
# print(targets)

In [ ]:
def train_loop_fun2(data_loader, model, optimizer, device, grad_accs, scheduler=None):
    model.train()
    t0 = time.time()
    losses = []
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        targets = [data["labels"] for data in batch]
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.stack(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids, lengt=lengt)
        loss = loss_fun(outputs, targets)
        # print('calcula la loss', loss)
        # Gradient accumulation loss
        (loss / grad_accs).backward()
        model.float()
        if (batch_idx + 1) % grad_accs == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
        losses.append(loss.item())
        if batch_idx % 250 == 0:
            print(
                f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} seconds ___")
            t0 = time.time()
    return losses

def individual_train_loop_fun2(data_loader, model, optimizer, device, grad_accs, scheduler=None, label_index=0):
    model.train()
    t0 = time.time()
    losses = []
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["labels"][label_index] for data in batch]
        targets = []
        if len(labels) > 1:
            for label_set in labels:
              miniset = []
              for label in label_set:
                miniset.append(torch.tensor([label]))
              targets.append(torch.stack(miniset))
        else:
            miniset = [torch.tensor([labels[0]])]
            targets.append(torch.stack(miniset))
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids, lengt=lengt)
        # outputs = torch.cat(torch.unbind(outputs))
        loss = loss_fun(outputs, targets)
        (loss / grad_accs).backward()
        model.float()
        if (batch_idx + 1) % grad_accs == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
        losses.append(loss.item())
        if batch_idx % 250 == 0:
            print(
                f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} seconds ___")
            t0 = time.time()
    return losses

In [ ]:
def eval_loop_fun2(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        #         model.half()
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        targets = [data["labels"] for data in batch]
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.stack(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        with torch.no_grad():
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids, lengt=lengt)
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())

        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

def individual_eval_loop_fun2(data_loader, model, device, label_index=0):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["labels"][label_index] for data in batch]
        targets = []
        if len(labels) > 1:
            for label_set in labels:
              miniset = []
              for label in label_set:
                miniset.append(torch.tensor([label]))
              targets.append(torch.stack(miniset))
        else:
            miniset = [torch.tensor([labels[0]])]
            targets.append(torch.stack(miniset))
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        with torch.no_grad():
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids, lengt=lengt)
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())

        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

In [ ]:
best_global_f1 = 0
best_global_model = 0
best_i = 0
# LABEL_INDEX = 3
learning_rates_results = {
      0: {
          "1_6": 0.5433715220949263,
          "5_6": 0.6085526315789473,
          "10_6": 0.555921052631579,
          "15_6": 0.5377049180327869,
          "20_6": "destroyed"
    },
      1: {
          "1_6": 0.5361842105263158,
          "5_6": 0.561038961038961,
          "10_6": 0.5846153846153846,
          "15_6": 0.5710659898477157,
          "20_6": "destroyed"
      },
      2: {
          "1_6": 0.2039473684210526,
          "5_6": 0.21533442088091354,
          "10_6": "semi-destroyed",
          "15_6": "destroyed"
      },
      3: {
          "1_6": 0.4785478547854785,
          "5_6": 0.43234323432343236,
          "10_6": 0.4422442244224422,
          "15_6": 0.429042904290429,
          "20_6": 0.42574257425742573,
          "25_6": 0.42574257425742573,
      },
        4: {
            "1_6": 0.3256578947368421,
            "5_6": 0.3288888888888889,
            "10_6": "destroyed"
        },
        5: {
            "1_6": 0.2401315789473684,
            "5_6": 0.23684210526315788,
            "10_6": 0.21994134897360704,
            "15_6": "destroyed"
        },
        6: {
            "1_6": 0.5155993431855501,
            "5_6": 0.5415986949429037,
            "10_6": 0.5237315875613748,
            "15_6": "semidestroyed",
            "20_6": "destroyed"
        },
        7: {
            "1_6": 0.023026315789473683,
            "5_6": "destroyed"
        },
        8: {
            "1_6": 0.04276315789473684,
            "5_6": 0.04276315789473684,
            "10_6": "destroyed"
        },
        9: {
            "1_6": 0.10180623973727422,
            "5_6": 0.0980392156862745,
            "10_6": 0.09733124018838306,
            "15_6": 0.09733124018838306,
            "20_6": "destroyed"
        },
      10: {
          "1_6": 0.04276315789473684,
          "5_6": 0.042414355628058724,
          "10_6": 0.042071197411003236,
          "15_6": 0.04220779220779221,
          "20_6": 0.042071197411003236,
          "25_6": "destroyed"
      },
      11: {
          "1_6": 0.5361842105263158,
          "5_6": 0.535303776683087,
          "10_6": 0.5686274509803922,
          "15_6": 0.5564648117839608,
          "20_6": 0.5328947368421052,
          "25_6": 0.5553914327917282
      },
      12: {
          "1_6": 0.3834710743801653,
          "5_6": 0.3531353135313531,
          "10_6": 0.3564356435643564,
          "15_6": 0.36363636363636365,
          "20_6": 0.38283828382838286,
          "25_6": "destroyed"
      },
      13: {
          "1_6": 0.17077175697865354,
          "5_6": 0.1731066460587326,
          "10_6": 0.17105263157894737,
          "15_6": "destroyed"
      }
}

learning_rates_coeffs = {
    0: 5,
    1: 10,
    2: 5,
    3: 1,
    4: 5,
    5: 1,
    6: 5,
    7: 1,
    8: 1,
    9: 1,
    10: 1,
    11: 10,
    12: 1,
    13: 5
}

for lab in MULTI_LABELS:
    LABEL_INDEX = lab
    print('#################################################### Para label: ', Y_KEYS[LABEL_INDEX], " ("+str(LABEL_INDEX)+") ##############")
    # i = 20
    # lrs = [1, 2, 3, 4, 5, 6, 7, 8 ,9 ,10, 15, 20, 25]

    i = learning_rates_coeffs[lab]
    print("------------------------------------------- exp: "+str(i)+"*1e-6 -----------------------------------------")
    lr=(i)*1e-6 # Algunos lr hacen que BERT se destruya 

    num_training_steps=int(len(transformed_train_dataset) / TRAIN_BATCH_SIZE * EPOCH)

    # Modify according to individual or all
    # if i > 0:
    #     del model
    #     del best_model
    model=Single_Bert_Hierarchical_Model().to(device) # Bert_FT_Model(len(Y_KEYS)).to(device)
    optimizer=AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                        num_warmup_steps=0,
                                        num_training_steps=num_training_steps)
    val_losses=[]
    batches_losses=[]
    val_acc=[]

    best_model = None
    best_g_mean = 0

    for epoch in range(EPOCH):
        t0 = time.time()    
        print(f"\n=============== EPOCH {epoch+1} / {EPOCH} ===============\n")
        # Modify according to individual or all
        batches_losses_tmp=individual_train_loop_fun2(train_data_loader, model, optimizer, device, GRADIENT_ACCUMULATIONS, scheduler=scheduler, label_index=LABEL_INDEX)
        epoch_loss=np.mean(batches_losses_tmp)
        print(f"\n*** avg_loss : {epoch_loss:.2f}, time : ~{(time.time()-t0)//60} min ({time.time()-t0:.2f} sec) ***\n")
        t1=time.time()
        # Modify according to individual or all
        output, target, val_losses_tmp=individual_eval_loop_fun2(val_data_loader, model, device, label_index=LABEL_INDEX)
        print(f"==> evaluation : avg_loss = {np.mean(val_losses_tmp):.2f}, time : {time.time()-t1:.2f} sec\n")
        # Modify according to individual or all
        tmp_evaluate=individual_evaluation(target, output)
        # if tmp_evaluate['f1'] > best_f1 and tmp_evaluate['recall'] != 0 and tmp_evaluate['specificity'] != 0:
        #   best_model = model
        #   best_f1 = tmp_evaluate['f1']
        if tmp_evaluate['g_mean'] > best_g_mean or tmp_evaluate['default_g_mean'] > best_g_mean:# and tmp_evaluate['recall'] != 0 and tmp_evaluate['specificity'] != 0:
            del best_model
            torch.cuda.empty_cache()
            best_model = model
            if tmp_evaluate['g_mean'] > tmp_evaluate['default_g_mean']:
                best_g_mean = tmp_evaluate['g_mean']
            else:
                best_g_mean = tmp_evaluate['default_g_mean']
        print(f"=====>\t{tmp_evaluate}")
        val_acc.append(tmp_evaluate['accuracy'])
        val_losses.append(val_losses_tmp)
        batches_losses.append(batches_losses_tmp)

    torch.save(best_model, f"/research/jamunoz/models/average_fine_tuned_bert/model_"+str(lab)+".pt")
    del model, optimizer, scheduler, best_model
    torch.cuda.empty_cache()

    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@") 
        



# lr=1e-3
# num_training_steps=int(len(transformed_train_dataset) / TRAIN_BATCH_SIZE * EPOCH)

# model=Bert_Hierarchical_Model(len(Y_KEYS)).to(device)
# optimizer=AdamW(model.parameters(), lr=lr)
# scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                         num_warmup_steps=0,
#                                         num_training_steps=num_training_steps)
# val_losses=[]
# batches_losses=[]
# val_acc=[]
# for epoch in range(EPOCH):
#     t0 = time.time()    
#     print(f"\n=============== EPOCH {epoch+1} / {EPOCH} ===============\n")
#     batches_losses_tmp=train_loop_fun2(train_data_loader, model, optimizer, device, GRADIENT_ACCUMULATIONS, scheduler=scheduler)
#     epoch_loss=np.mean(batches_losses_tmp)
#     print(f"\n*** avg_loss : {epoch_loss:.2f}, time : ~{(time.time()-t0)//60} min ({time.time()-t0:.2f} sec) ***\n")
#     t1=time.time()
#     output, target, val_losses_tmp=eval_loop_fun2(val_data_loader, model, device)
#     print(f"==> evaluation : avg_loss = {np.mean(val_losses_tmp):.2f}, time : {time.time()-t1:.2f} sec\n")
#     tmp_evaluate=evaluate(target, output)
#     print(f"=====>\t{tmp_evaluate}")
#     val_acc.append(tmp_evaluate['accuracy'])
#     val_losses.append(val_losses_tmp)
#     batches_losses.append(batches_losses_tmp)
#     print("\t§§ model has been saved §§")
#     # torch.save(model, f"gdrive/My Drive/magister/models/fine_tuned_bert_hierarchical_mean/model_epoch{epoch+1}.pt")

#################################################### Para label:  Apoyo pedagógico personal  (1) ##############
------------------------------------------- exp: 10*1e-6 -----------------------------------------


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<ipython-input-18-5fb9fe6293fb>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  remain = torch.tensor(remain, dtype=torch.long)



=============== EPOCH 1 / 6 ===============

___ batch index = 0 / 1836 (0.00%), loss = 0.6546, time = 0.08 seconds ___


___ batch index = 250 / 1836 (13.62%), loss = 0.6281, time = 29.13 seconds ___


...

___ batch index = 1750 / 1836 (95.32%), loss = 0.0578, time = 29.79 seconds ___



*** avg_loss : 0.35, time : ~3.0 min (208.58 sec) ***



==> evaluation : avg_loss = 0.32, time : 28.62 sec



threshold:  [0.04560347]
=====>	{'Positive Rate': array([0.092257], dtype=float32), 'accuracy': 0.1614497528830313, 'f1': 0.1696574225122349, 'recall': 0.9285714285714286, 'specificity': 0.08348457350272233, 'g_mean': 0.27842663249247424, 'default_accuracy': 0.9077429983525536, 'default_f1': 0.0, 'default_recall': 0.0, 'default_specificity': 1.0, 'default_g_mean': 0.0}


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
